In [1]:
import os 

import pandas as pd
import numpy as np
import feather



In [2]:
fp = os.getcwd().replace('Models','')


In [3]:
train=feather.read_dataframe(fp + 'data/train.feather')


In [5]:
test=feather.read_dataframe(fp + 'data/test.feather')


In [8]:
import datetime
import time
from collections import defaultdict
import gc
import numpy as np

In [9]:
# #transform target variable to log(1 + demand)  

train['Demanda_uni_equil']=np.log1p(train['Demanda_uni_equil'])

In [10]:
cpa_arr = defaultdict(int)
cpa_count = defaultdict(int)
pr_arr = defaultdict(int)
pr_count = defaultdict(int)
product_arr = defaultdict(int)
product_count = defaultdict(int)

In [11]:
week=train['Semana'].values
agency=train['Agencia_ID'].values
canal_id=train['Canal_ID'].values
ruta_sak=train['Ruta_SAK'].values
cliente_id=train['Cliente_ID'].values
producto_id=train['Producto_ID'].values
vuh=train['Venta_uni_hoy'].values
vh=train['Venta_hoy'].values
dup=train['Dev_uni_proxima'].values
dp = train['Dev_proxima'].values
target=train['Demanda_uni_equil'].values

In [12]:
zip_cpa=zip(cliente_id,producto_id,agency)

In [13]:
zip_pr=zip(producto_id,ruta_sak)

In [14]:
avg_target= 0.0
total=len(target)
print len(target)
for i in range(len(target)):
    
    cpa_arr[zip_cpa[i]]+=target[i]
    pr_arr[zip_pr[i]] +=target[i]
    product_arr[producto_id[i]] += target[i]
     
    
    cpa_count[zip_cpa[i]]+=1
    pr_count[zip_pr[i]] +=1
    product_count[producto_id[i]] +=1
    avg_target += target[i]

     

74180464


In [15]:
avg_target/=total
mylist=[]

In [16]:
for row in test.itertuples():
    
    cpa_test=(row[6],row[7],row[3])
    pr_test=(row[7],row[5])
    product_test=row[7]
 
    if cpa_test in cpa_arr:
        value=cpa_arr[cpa_test]/cpa_count[cpa_test]
    elif pr_test in pr_arr:
        value=pr_arr[pr_test]/pr_count[pr_test]
    elif product_test in product_arr:
        value=product_arr[product_test]/product_count[product_test]
    else:
        value=avg_target
    mylist.append(value)

In [25]:
mylist_exp=np.expm1(mylist)

In [26]:
submission = test.drop(['Semana','Agencia_ID','Canal_ID','Ruta_SAK','Cliente_ID','Producto_ID'],1)

In [27]:
submission['Demanda_uni_equil']=mylist_exp

In [29]:
submission['Demanda_uni_equil']=submission['Demanda_uni_equil']*0.9

In [30]:
submission.to_csv('submission710_1933.csv',index=False)

In [32]:
submission

,id,Demanda_uni_equil
0,0,3.600000
1,1,1.562350
2,2,1.800000
3,3,1.696673
4,4,2.438058
5,5,2.977982
6,6,1.160486
7,7,5.073316
8,8,5.161710
9,9,5.445591
